## Snowflake SQL API 시작하기

https://quickstarts.snowflake.com/guide/getting_started_snowflake_sql_api_kr/index.html?index=..%2F..kr#0

## API 소개


다음 URL의 버전을 탐색하여 SQL API로 이동합니다. 여러분의 Snowflake 계정을 위한 계정 로케이터로 *account_locator*를 대체합니다.

> https://*account_locator*.snowflakecomputing.com/api/v2


사용을 시작하기 전에 이제 API의 각 부분을 분석해 보겠습니다. API는 /api/v2/statements/ 리소스로 구성되어 있으며 다음 엔드포인트를 제공합니다.

* `/api/v2/statements`
    * 이 엔드포인트는 실행을 위한 SQL 문을 제출하기 위해 사용합니다.
* `/api/v2/statements/*statementHandle*`
    * 이 엔드포인트는 문의 실행 상태를 확인하기 위해 사용합니다.
* `/api/v2/statements/*statementHandle*/cancel`
    * 이 엔드포인트는 문의 실행을 취소하기 위해 사용합니다.
    * 다음 단계에서는 이러한 모든 엔드포인트를 사용하여 여러분이 API에 익숙해질 수 있도록 합니다.

### SQL API의 한계

* 현존하는 SQL API의 한계를 인지하고 있는 것이 중요합니다. 특히나 GET 및 PUT은 지원되지 않습니다.

## 요청 재제출을 위해 고유한 요청 ID 할당

일부 경우 Snowflake가 API 요청에서 SQL 문을 실행했는지가 확실하지 않을 수 있습니다(예: 네트워크 오류 또는 시간제한으로 인해). Snowflake가 문을 실행하지 않았을 경우 Snowflake에 동일한 요청을 재제출하기로 결정할 수 있습니다.

Snowflake가 이미 초기 요청에서 문을 실행했는데 여러분이 요청을 재제출한다면 문은 두 번 실행됩니다. 일부 요청 유형의 경우 반복적으로 동일한 문을 실행하면 의도하지 않은 결과(예: 테이블에 중복 데이터 삽입)가 나타날 수 있습니다.

여러분이 요청을 재제출할 때 Snowflake가 동일한 문을 두 번 실행하는 것을 방지하기 위해 요청 ID를 사용하여 여러분의 요청을 다른 요청과 구별할 수 있습니다. 여러분이 초기 실행과 재제출 요청에 동일한 요청 ID를 지정했다고 가정하겠습니다. 이러한 경우 해당 문이 이미 성공적으로 실행되었다면 Snowflake는 문을 다시 실행하지 않습니다.

요청 ID를 지정하려면 UUID(Universally Unique Identifier)를 생성하고 이 식별자를 requestId 쿼리 매개 변수에 포함합니다.

> POST /api/v2/statements?requestId=<UUID> HTTP/1.1

Snowflake가 요청을 처리하는 데 실패하면 동일한 요청 ID로 동일한 요청을 다시 제출할 수 있습니다. 동일한 요청 ID를 사용하는 것은 서버에 여러분이 동일한 요청을 다시 제출하고 있음을 알립니다.

이제 요청에 포함해야 하는 추가 정보인 인증 매개 변수에 대해 알아보겠습니다.

## 서버에 인증

요청을 전송할 때 요청에는 인증 정보가 반드시 포함되어야 합니다. 인증을 제공하는 2가지 옵션은 OAuth 및 JWT 키 쌍 인증입니다. 둘 중 이전에 구현했던 것이나 가장 익숙한 것을 사용할 수 있습니다. 이 예에서는 JWT로 인증하는 것을 상세하게 설명합니다.

아직 준비하지 않았다면 Snowflake와 이미 작동하고 있는 키 쌍 인증을 준비하십시오.

다음 명령을 사용하여 Snowflake 키 쌍과 성공적으로 연결할 수 있는지 확인하기 위해 테스트할 수 있습니다.

> snowsql -a <account> -u <user> --private-key-path <path to private key>

Negative : OAuth로 인증하고자 한다면 OAuth로 인증하는 것에 대한 Snowflake 설명서를 방문하십시오.

키 쌍 인증을 사용하여 Snowflake와 연결할 수 있음을 확인한 후 JWT 토큰을 생성해야 합니다. 이 JWT 토큰은 여러분의 키로 서명된 시간제한 토큰입니다. 또한 Snowflake는 여러분이 SQL API를 위해 이 토큰을 인증에 사용할 수 있도록 인가했음을 알게 됩니다.

> snowsql -a <account> -u <user> --private-key-path <path to private key> --generate-jwt
<returns JWT token>

생성된 JWT 토큰이 SQL API 사용에 사용되어야 합니다. 다음 헤더는 여러분의 애플리케이션 코드 내에서 전송하는 각 API 요청에 설정되어야 합니다.

* `*jwt_token*` 이 SnowSQL에서 생성된 JWT 토큰인 경우 Authorization: `Bearer *jwt_token*`
* `X-Snowflake-Authorization-Token-Type`: `KEYPAIR_JWT`

전체적으로 여러분의 요청 쿼리와 헤더는 다음과 같은 형식입니다.

```

POST /api/v2/statements?requestId=<UUID> HTTP/1.1
Authorization: Bearer <jwt_token>
Content-Type: application/json
Accept: application/json
User-Agent: myApplication/1.0
X-Snowflake-Authorization-Token-Type: KEYPAIR_JWT

```

## SQL 문 실행을 위해 요청 제출

실행을 위해 SQL 문을 제출하려면 POST 요청을 `/api/v2/statements/ endpoint`로 전송합니다.

```
POST /api/v2/statements?requestId=<UUID> HTTP/1.1
Authorization: Bearer <jwt_token>
Content-Type: application/json
Accept: application/json
User-Agent: myApplication/1.0
X-Snowflake-Authorization-Token-Type: KEYPAIR_JWT

(request body)

```
요청 URL에서 쿼리 매개 변수를 다음과 같이 설정할 수 있습니다.

* 비동기로 문 실행: async=true

요청 본문을 위해 다음 필드를 설정합니다.

* 실행하고 싶은 SQL 문에 statement 필드를 설정합니다.
* 웨어하우스, 데이터베이스, 스키마 및 역할을 사용에 지정하기 위해 warehouse, database, schema 및 role 필드를 설정합니다.
N
egative : 참고: 이러한 필드에 있는 값은 대/소문자를 구분합니다.

* 문 실행에 시간제한을 설정하려면 timeout 필드를 최대 대기 시간(초)으로 설정합니다. timeout 필드가 설정되어 있지 않다면 STATEMENT_TIMEOUT_IN_SECONDS 매개 변수에 의해 지정된 시간제한이 사용됩니다.


```

POST /api/v2/statements HTTP/1.1
Authorization: Bearer <jwt_token>
Content-Type: application/json
Accept: application/json
User-Agent: myApplication/1.0
X-Snowflake-Authorization-Token-Type: KEYPAIR_JWT

{
    "statement": "select * from T",
    "timeout": 60,
    "database": "<your_database>",
    "schema": "<your_schema>",
    "warehouse": "<your_warehouse>",
    "role": "<your_role>"
}

```

이 요청의 몇몇 특정 필드를 알아보겠습니다.

* statement 필드는 실행할 SQL 문을 지정합니다.
* timeout 필드는 서버가 문 실행 시간을 60초까지 허용하도록 지정합니다.

문이 성공적으로 실행되었다면 Snowflake는 HTTP 응답 코드 200을 반환하며 첫 결과를 ResultSet 오브젝트로 반환합니다. 바인드 변수를 포함하는 방법을 알아본 다음 상태를 확인하고 결과를 검색하는 방법을 알아보겠습니다.

이제 바인드 변수(? 자리 표시자)를 문에 포함하고 각 변수에 해당하는 Snowflake 데이터 형식과 값을 지정하는 오브젝트에 bindings 필드를 설정할 수 있는 방법을 알아보겠습니다.

> 이하 생략